In [1]:
import os

In [2]:
%pwd

'c:\\Users\\rohan\\Desktop\\Machine Learning Projects\\Test summirization\\Text-Summarization\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\rohan\\Desktop\\Machine Learning Projects\\Test summirization\\Text-Summarization'

In [5]:
# %pip install --upgrade accelerate
# %pip uninstall -y transformers accelerate
# %pip install transformers accelerate

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epoch: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_step: int
    evalution_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_step: int

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directory

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directory([self.config.artifacts_root])
    
    def get_model_trainer_config(self) -> ModelTrainerConfig:
          config = self.config.model_trainer
          params = self.params.TrainingArguments

          create_directory([config.root_dir])

          model_trainer_config = ModelTrainerConfig(
              root_dir=config.root_dir,
              data_path=config.data_path,
              model_ckpt=config.model_ckpt,
              num_train_epoch=params.num_train_epoch,
              warmup_steps=params.warmup_steps,
              per_device_train_batch_size=params.per_device_train_batch_size,
              weight_decay=params.weight_decay,
              logging_step=params.logging_step,
              evalution_strategy=params.evalution_strategy,
              eval_steps=params.eval_steps,
              save_steps=params.save_steps,
              gradient_accumulation_step=params.gradient_accumulation_step
          )
          return model_trainer_config

In [9]:
#pip install tf-keras

In [10]:
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer
from datasets import load_dataset, load_from_disk
import torch

[2024-08-09 21:59:00,881: INFO: utils: NumExpr defaulting to 8 threads.]
[2024-08-09 21:59:07,943: WARNING: module_wrapper: From C:\Users\rohan\AppData\Roaming\Python\Python39\site-packages\tf_keras\src\losses.py:2976: The name tf.losses.sparse_softmax_cross_entropy is deprecated. Please use tf.compat.v1.losses.sparse_softmax_cross_entropy instead.
]
[2024-08-09 21:59:08,667: INFO: config: PyTorch version 2.3.0 available.]
[2024-08-09 21:59:08,667: INFO: config: Polars version 0.20.25 available.]
[2024-08-09 21:59:08,667: INFO: config: TensorFlow version 2.17.0 available.]


In [11]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config


    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model = model_pegasus)

        #loading dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            output_dir=self.config.root_dir, num_train_epochs=self.config.num_train_epoch, warmup_steps=self.config.warmup_steps,
            per_device_train_batch_size=self.config.per_device_train_batch_size, per_device_eval_batch_size=self.config.per_device_train_batch_size,
            weight_decay=self.config.weight_decay, logging_steps=self.config.logging_step,
            evaluation_strategy=self.config.evalution_strategy, eval_steps=self.config.eval_steps, save_steps=1e6,
            gradient_accumulation_steps=self.config.gradient_accumulation_step
        ) 

        # trainer_args = TrainingArguments(
        #     output_dir=self.config.root_dir, num_train_epochs=1, warmup_steps=500,
        #     per_device_train_batch_size=1, per_device_eval_batch_size=1,
        #     weight_decay=0.01, logging_steps=10,
        #     evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
        #     gradient_accumulation_steps=16
        #     ) 
        trainer = Trainer(
            model=model_pegasus,
            args=trainer_args,
            tokenizer = tokenizer,
            data_collator = seq2seq_data_collator,
            train_dataset=dataset_samsum_pt["test"],
            eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))


In [12]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-08-09 21:59:09,188: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-08-09 21:59:09,198: INFO: common: yaml file: params.yaml loaded successfully]
[2024-08-09 21:59:09,202: INFO: common: created directory at: artifacts]
[2024-08-09 21:59:09,204: INFO: common: created directory at: artifacts/model_trainer]


C:\Users\rohan\AppData\Roaming\Python\Python39\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\rohan\AppData\Roaming\Python\Python39\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.w

  0%|          | 0/51 [00:00<?, ?it/s]

KeyboardInterrupt: 